# INCA Parser - Quick start
The INCA parser introduce a python wrapper to the Matlab application INCA. INCA is capable of conducting multiple different 13C-MFA algorithms, but it meant to be used through a GUI and there it is hard in incorporate into high throughput pipelines or just laborious to handle many samples and measurements. The INCA parser operates on both sides of INCA it helps input model and data into INCA, it can run INCA algorithms and finally provides a workflow to export data from INCA after execution of algorithms.

In [11]:
import pandas as pd
import pathlib
from BFAIR.mfa.INCA.INCA_script import INCA_script
from BFAIR.mfa.INCA.INCA_script_writing import define_options, define_runner, create_inca_script_from_data
import ast
from BFAIR.mfa.INCA.run_inca import run_inca


The input data to the INCA parser is given as pandas dataframes, that has to comply to specified dataschemas (See XX for a thorough walk through of the requirements). The minimum inputs to run MFA in INCA is 
- Reaction data
- Tracer data

For stationary 13C-MFA least one of
- Flux measurements
- MS measurements

For Isotopically Non-stationary 13C-MFA additionally measurements of the metabolite pool are required.

Let's load the data

In [12]:
data_folder = pathlib.Path("../../tests/test_data/MFA_modelInputsData/simple_model/")
tracers_data = pd.read_csv(data_folder / "tracers.csv", 
   converters={'atom_mdv':ast.literal_eval}
)
reactions_data = pd.read_csv(data_folder / "reactions.csv")
flux_data = pd.read_csv(data_folder / "measuredFluxes.csv")
ms_data = pd.read_csv(data_folder / "experimentalMS_data_multiple_experiments.csv", 
   converters={'atom_ids':ast.literal_eval, 'labelled_atom_ids': ast.literal_eval, 'idv': ast.literal_eval, 'idv_std_error': ast.literal_eval}
)

We use a simple toy model with 5 reactions [1,2]. In this data set we have two experiments exp1 and exp2. They were conducted with different tracers. For exp1 we have both flux and MS measurements, while for exp2 the data set only contains MS measurements. We will quick view the input data, again for a detail description see XX.

In [13]:
reactions_data.head()

,model,rxn_id,rxn_eqn
0,simple_model,R1,A (abc) -> B (abc)
1,simple_model,R2,B (abc) <-> D (abc)
2,simple_model,R3,B (abc) -> C (bc) + E (a)
3,simple_model,R4,B (abc) + C (de) -> D (bcd) + E (a) + E (e)
4,simple_model,R5,D (abc) -> F (abc)


In [14]:
tracers_data.head()

,experiment_id,met_id,tracer_id,atom_ids,ratio,atom_mdv,enrichment
0,exp1,A,[1-13C]A,[1],1.0,"[0, 1]",1
1,exp2,A,"[1,2-13C]A","[1,2]",0.5,"[0.05, 0.95]",1


In [15]:
flux_data.head()

,experiment_id,rxn_id,flux,flux_std_error
0,exp1,R1,10.0,0.00001


In [16]:
ms_data.head()

,experiment_id,time,met_id,ms_id,unlabelled_atoms,labelled_atom_ids,idv,idv_std_error
0,exp1,0,F,F1,C3H5O1,"[1, 2, 3]","[0.01, 0.8, 0.1, 0.0009]","[0.0003, 0.003, 0.0008, 0.001]"
1,exp2,0,F,F1,C3H5O1,"[1, 2, 3]","[0.5, 0.45, 0.05, 0.0]","[3e-05, 0.0027, 0.03, 0.001]"


With the data loaded we are ready to create the INCA script that specifies the model, data and algorithms that we wish to apply. We will specify that estimation algorithm to do 5 restarts and that INCA should not do naturel abundance correction. Finally, specify that we wish to run the three of INCA's algorithms: estimate, simulate and continuate and that the model including the results should be saved in a file called simple_model_quickstart.mat in the current working directory.

In [19]:
output_file = "simple_model_quikstart.mat"
script = create_inca_script_from_data(reactions_data, tracers_data, flux_data, ms_data)
script.add_to_block("options", define_options(fit_starts=5,sim_na=False))
script.add_to_block("runner", define_runner(output_file, run_estimate=True, run_simulation=True, run_continuation=True))

Now the script is ready to run in matlab.

In [23]:
import dotenv
inca_directory = pathlib.Path(dotenv.get_key(dotenv.find_dotenv(), "INCA_base_directory"))
run_inca(script, INCA_base_directory=inca_directory)

INCA script saved to /var/folders/z6/mxpxh4k56tv0h0ff41vmx7gdwtlpvp/T/tmpvzylx33u/inca_script.m.
Starting MATLAB engine...


Error using regexp
Invalid option for regexp: .

Error in tracer>subparse

Error in tracer>parse

Error in tracer

Error in inca_script (line 15)
t_exp1 = tracer({...



MatlabExecutionError: 
  File /Users/s143838/inca2.1/class/@tracer/tracer.p, line 0, in subparse

  File /Users/s143838/inca2.1/class/@tracer/tracer.p, line 0, in parse

  File /Users/s143838/inca2.1/class/@tracer/tracer.p, line 0, in tracer

  File /private/var/folders/z6/mxpxh4k56tv0h0ff41vmx7gdwtlpvp/T/tmpvzylx33u/inca_script.m, line 15, in inca_script
Invalid option for regexp: .


In [22]:
dotenv.get_key(dotenv.find_dotenv(), "INCA_directory")